In [5]:
!pwd

/Users/pez/DataspellProjects/music-mining/notebooks/collection


In [6]:
import json

import pandas as pd

from util import fetch_bulk_track_features, fetch_spotify_chart,  extract_track_ids,\
    enrich_with_artist_nominations_summary, with_mbz_artist_metadata

In [7]:
song_chart_df = pd.read_parquet("../../data/raw/song_chart_summary.pq")

spotify_chart_df = fetch_spotify_chart(seed=42)
spotify_chart_df["id"] = spotify_chart_df["url"].apply(lambda url: url.split("/")[-1])
# maybe also incldue chart from 2017?

recording_nominations_df = (
    pd.read_parquet("../../data/raw/recording_nominations.pq")[[
        "award_category",
        "award_nominee",
        "award_winner",
        "spotify_track_id",
    ]]
    .groupby(["spotify_track_id"])
    .aggregate({
        "award_category": "max",
        "award_nominee": "sum",
        "award_winner": "sum",
    })
    .reset_index()
    .drop_duplicates(subset=["spotify_track_id"])
    .rename(columns={
        "award_category": "recording_award_category",
        "award_nominee": "recording_award_nominee",
        "award_winner": "recording_award_winner",
    })
)

recording_nominations_track_ids = extract_track_ids(recording_nominations_df, "spotify_track_id")
song_chart_track_ids = extract_track_ids(song_chart_df, "id")
spotify_chart_track_ids = extract_track_ids(spotify_chart_df, "id")
track_ids = list(set(recording_nominations_track_ids + song_chart_track_ids\
                     + spotify_chart_track_ids))
track_features = fetch_bulk_track_features(track_ids)
track_features_df = pd.DataFrame(track_features)
track_features_df = pd.merge(track_features_df, recording_nominations_df,
                             left_on="id", right_on="spotify_track_id", how="outer")
track_features_df = pd.merge(track_features_df, song_chart_df,
                             on=["id", "album_id"], how="outer")

track_features_df

fetching track metadata: 0
fetching track audio features: 0
fetching track metadata: 50
fetching track audio features: 50
fetching track metadata: 100
fetching track audio features: 100
fetching track metadata: 150
fetching track audio features: 150
fetching track metadata: 200
fetching track audio features: 200
fetching track metadata: 250
fetching track audio features: 250
fetching track metadata: 300
fetching track audio features: 300
fetching track metadata: 350
fetching track audio features: 350
fetching track metadata: 400
fetching track audio features: 400
fetching track metadata: 450
fetching track audio features: 450
fetching track metadata: 500
fetching track audio features: 500
fetching track metadata: 550
fetching track audio features: 550
fetching track metadata: 600
fetching track audio features: 600
fetching track metadata: 650
fetching track audio features: 650
fetching track metadata: 700
fetching track audio features: 700
fetching track metadata: 750
fetching track au

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,spotify_track_id,recording_award_category,recording_award_nominee,recording_award_winner,artist_id,months,indicativerevenue,position_max,position_mean,position_median
0,2egI1hFZDAYrkCgtepoore,Cristina (feat. Shelow Shaq),Cristina (feat. Shelow Shaq),2ZTlbmxYIAhs8ckvgx0imI,"[""Maffio"", ""Justin Quiles"", ""Nacho"", ""Shelow S...","[""5RzT7CM6Ot0sh0EHefMicV"", ""14zUHaJZo1mnYtn6IB...",1,1,False,186893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2JoJrsEV15OzbijS47lids,Today Was A Fairytale (Taylor’s Version),Fearless (Taylor's Version),4hDok0OAJd57SGIT8xuWJH,"[""Taylor Swift""]","[""06HL4z0CvFAxyc27GXpf02""]",20,1,False,241822,...,NaN,NaN,NaN,NaN,06HL4z0CvFAxyc27GXpf02,3.0,1505.781,45.0,37.0,35.0
2,588RGyR1f4Plc8nm7SrdCa,Karma,The Diary Of Alicia Keys,6TqRKHLjDu5QZuC8u5Woij,"[""Alicia Keys""]","[""3DiDSECUqqY1AuBP8qtaIa""]",2,1,False,256000,...,NaN,NaN,NaN,NaN,3DiDSECUqqY1AuBP8qtaIa,5.0,6514.294,39.0,33.2,33.0
3,41CgzGD7xlgnJe14R4cqkL,Paris in the Rain,I met you when I was 18. (the playlist),7lqieLm717j9Z8JrG7POkS,"[""Lauv""]","[""5JZ7CnR6gTvEMKX4g70Amv""]",2,1,False,204712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0aBKFfdyOD1Ttvgv0cfjjJ,More - RedOne Jimmy Joker Remix,More,5GNPZT1Bxq0EP7PQDQmK3U,"[""Usher""]","[""23zg3TcAtWQy7J6upgbUnj""]",1,1,False,219986,...,NaN,NaN,NaN,NaN,23zg3TcAtWQy7J6upgbUnj,5.0,4789.541,37.0,21.2,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4457,0uz6Hh98QGCgP1BQ4KoW3k,"Step One, Step Two",CO-VID,6fUi7Hjb78o5Oy159sXpDl,"[""Unknown"", ""Teyy""]","[""2Sde8WG5zbgPdv0VknOzqt"", ""1D4YdZXEFPL1Humjke...",13,1,True,156238,...,NaN,NaN,NaN,NaN,2Sde8WG5zbgPdv0VknOzqt,3.0,2637.847,45.0,42.0,41.0
4458,6jDsiTA9jwNEMYU24bRFAW,Through The Eyes Of A Child - Live,Reamonn Live,7aYvBhx3XsoDRoEHTnwHth,"[""Reamonn""]","[""4VH2QabEORUUhN1F1cYQ6N""]",3,1,False,248026,...,NaN,NaN,NaN,NaN,4VH2QabEORUUhN1F1cYQ6N,1.0,864.744,36.0,36.0,36.0
4459,2mKouqwAIdQnMP43zxR89r,What's Luv? (feat. Ja-Rule & Ashanti),Jealous Ones Still Envy (J.O.S.E),6vMgb2mih3pPiWUxF8VEdI,"[""Fat Joe"", ""Ja Rule"", ""Ashanti""]","[""3ScY9CQxNLQei8Umvpx5g6"", ""1J2VVASYAamtQ3Bt8w...",9,1,True,267093,...,NaN,NaN,NaN,NaN,3ScY9CQxNLQei8Umvpx5g6,6.0,16836.322,26.0,10.0,7.0
4460,5ecZWU5uQOiCVSnPxBZNmT,7 Things - Single Version,Breakout,0Yu3czJNOQ68fZgkvpjuHL,"[""Miley Cyrus""]","[""5YGY8feqx7naU7z4HrwZM6""]",2,1,False,213453,...,NaN,NaN,NaN,NaN,5YGY8feqx7naU7z4HrwZM6,2.0,1876.356,29.0,28.5,28.5


In [8]:
album_nominations_df = (
    pd.read_parquet("../../data/raw/album_nominations.pq")[[
        "award_category",
        "award_nominee",
        "award_winner",
        "spotify_album_id",
    ]]
    .groupby(["spotify_album_id"])
    .aggregate({
        "award_category": "max",
        "award_nominee": "sum",
        "award_winner": "sum",
    })
    .reset_index()
    .drop_duplicates(subset=["spotify_album_id"])
    .rename(columns={
        "award_category": "album_award_category",
        "award_nominee": "album_award_nominee",
        "award_winner": "album_award_winner",
    })
)

df = (
    pd.merge(track_features_df, album_nominations_df,
             left_on="album_id", right_on="spotify_album_id", how="left")
        .drop(columns=["spotify_album_id"])
)
df

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,duration_ms,...,recording_award_winner,artist_id,months,indicativerevenue,position_max,position_mean,position_median,album_award_category,album_award_nominee,album_award_winner
0,2egI1hFZDAYrkCgtepoore,Cristina (feat. Shelow Shaq),Cristina (feat. Shelow Shaq),2ZTlbmxYIAhs8ckvgx0imI,"[""Maffio"", ""Justin Quiles"", ""Nacho"", ""Shelow S...","[""5RzT7CM6Ot0sh0EHefMicV"", ""14zUHaJZo1mnYtn6IB...",1,1,False,186893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2JoJrsEV15OzbijS47lids,Today Was A Fairytale (Taylor’s Version),Fearless (Taylor's Version),4hDok0OAJd57SGIT8xuWJH,"[""Taylor Swift""]","[""06HL4z0CvFAxyc27GXpf02""]",20,1,False,241822,...,NaN,06HL4z0CvFAxyc27GXpf02,3.0,1505.781,45.0,37.0,35.0,NaN,NaN,NaN
2,588RGyR1f4Plc8nm7SrdCa,Karma,The Diary Of Alicia Keys,6TqRKHLjDu5QZuC8u5Woij,"[""Alicia Keys""]","[""3DiDSECUqqY1AuBP8qtaIa""]",2,1,False,256000,...,NaN,3DiDSECUqqY1AuBP8qtaIa,5.0,6514.294,39.0,33.2,33.0,Best R&B Album,1.0,1.0
3,41CgzGD7xlgnJe14R4cqkL,Paris in the Rain,I met you when I was 18. (the playlist),7lqieLm717j9Z8JrG7POkS,"[""Lauv""]","[""5JZ7CnR6gTvEMKX4g70Amv""]",2,1,False,204712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0aBKFfdyOD1Ttvgv0cfjjJ,More - RedOne Jimmy Joker Remix,More,5GNPZT1Bxq0EP7PQDQmK3U,"[""Usher""]","[""23zg3TcAtWQy7J6upgbUnj""]",1,1,False,219986,...,NaN,23zg3TcAtWQy7J6upgbUnj,5.0,4789.541,37.0,21.2,17.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4457,0uz6Hh98QGCgP1BQ4KoW3k,"Step One, Step Two",CO-VID,6fUi7Hjb78o5Oy159sXpDl,"[""Unknown"", ""Teyy""]","[""2Sde8WG5zbgPdv0VknOzqt"", ""1D4YdZXEFPL1Humjke...",13,1,True,156238,...,NaN,2Sde8WG5zbgPdv0VknOzqt,3.0,2637.847,45.0,42.0,41.0,NaN,NaN,NaN
4458,6jDsiTA9jwNEMYU24bRFAW,Through The Eyes Of A Child - Live,Reamonn Live,7aYvBhx3XsoDRoEHTnwHth,"[""Reamonn""]","[""4VH2QabEORUUhN1F1cYQ6N""]",3,1,False,248026,...,NaN,4VH2QabEORUUhN1F1cYQ6N,1.0,864.744,36.0,36.0,36.0,NaN,NaN,NaN
4459,2mKouqwAIdQnMP43zxR89r,What's Luv? (feat. Ja-Rule & Ashanti),Jealous Ones Still Envy (J.O.S.E),6vMgb2mih3pPiWUxF8VEdI,"[""Fat Joe"", ""Ja Rule"", ""Ashanti""]","[""3ScY9CQxNLQei8Umvpx5g6"", ""1J2VVASYAamtQ3Bt8w...",9,1,True,267093,...,NaN,3ScY9CQxNLQei8Umvpx5g6,6.0,16836.322,26.0,10.0,7.0,NaN,NaN,NaN
4460,5ecZWU5uQOiCVSnPxBZNmT,7 Things - Single Version,Breakout,0Yu3czJNOQ68fZgkvpjuHL,"[""Miley Cyrus""]","[""5YGY8feqx7naU7z4HrwZM6""]",2,1,False,213453,...,NaN,5YGY8feqx7naU7z4HrwZM6,2.0,1876.356,29.0,28.5,28.5,NaN,NaN,NaN


In [9]:
def enrich_with_artist_nominations_summary(df, artist_nominations_df):
    skip_album_ids = []

    for index, track in df.iterrows():
        print(track["name"], track["artists"])
        if track["album_id"] in skip_album_ids:
            continue
        else:
            skip_album_ids.append(track["album_id"])
        track_ids = json.loads(track["artist_ids"].replace("'", "\""))
        track_release_year = track["release_date"][0:4]
        total_nominations = 0
        total_wins = 0
        first_nomination = []
        first_win = []
        csum_nominations = 0
        csum_wins = 0
        for artist_id in track_ids:
            an_ = artist_nominations_df[artist_nominations_df["spotify_artist_id"] == artist_id]
            if len(an_) > 0:
                an_ = an_.to_dict('records')[0]
                total_nominations += an_["award_nominee"]
                total_wins += an_["award_winner"]
                first_nomination.append(an_["first_nomination"])
                first_win.append(an_["first_win"])
                if track_release_year:
                    for year in range(1995, int(track_release_year)):
                        csum_nominations += an_[f"nominated_{year}"]
                        csum_wins += an_[f"won_{year}"]
        df.loc[df["album_id"] == track["album_id"],
               "artist_total_nominations"] = an_["award_nominee"]
        df.loc[df["album_id"] == track["album_id"],
               "artist_total_wins"] = an_["award_winner"]
        df.loc[df["album_id"] == track["album_id"],
               "artist_first_nomination"] = an_["first_nomination"]
        df.loc[df["album_id"] == track["album_id"],
               "artist_first_win"] = an_["first_win"]
        df.loc[df["album_id"] == track["album_id"],
               "artist_previous_nominations"] = csum_nominations
        df.loc[df["album_id"] == track["album_id"],
               "artist_previous_wins"] = csum_wins

    return df


artist_nominations_df = pd.read_parquet("../../data/raw/artist_nominations_summary.pq")
df = enrich_with_artist_nominations_summary(df, artist_nominations_df)

Cristina (feat. Shelow Shaq) ["Maffio", "Justin Quiles", "Nacho", "Shelow Shaq"]
Today Was A Fairytale (Taylor’s Version) ["Taylor Swift"]
Karma ["Alicia Keys"]
Paris in the Rain ["Lauv"]
More - RedOne Jimmy Joker Remix ["Usher"]
Better Dig Two ["The Band Perry"]
Even If It Breaks Your Heart ["Eli Young Band"]
Ponte Pa' Mi ["Justin Quiles"]
Before I Forget ["Slipknot"]
Love Story (Taylor’s Version) ["Taylor Swift"]
4 Minutes (feat. Justin Timberlake & Timbaland) ["Madonna", "Justin Timberlake", "Timbaland"]
Blame It (Made Famous by Jamie Foxx & T Pain) ["The Animal Kingdom"]
Loyalty ["2017 Dynamo Hitz"]
Cock Cakes ["Bart Baker"]
Fat Lip ["Sum 41"]
100 Years (One Hundred Years) [In the Style of Five for Fighting] - Instrumental Only ["ProSource Karaoke"]
We Can't Stop ["Miley Cyrus"]
I Want Crazy - Encore ["Hunter Hayes"]
Believe ["Mumford & Sons"]
Got 2 Luv U (Tribute to Sean Paul & Alexis Jordan) ["Cover Pop"]
I Kissed A Girl ["Katy Perry"]
Vermissen (feat. Henning May) ["Juju", "Henn

In [10]:
artist_nominations_df

,artist_mbid,spotify_artist_id,artist_name,award_nominee,award_winner,nominated_1995,won_1995,nominated_1996,won_1996,nominated_1997,...,won_2019,nominated_2020,won_2020,nominated_2021,won_2021,first_win,last_win,first_nomination,last_nomination,spotify_popularity
0,0031bc7a-1b6f-4620-99db-5202405b0749,2Irt3HB3JdzLWsDmvB9QP6,Lena Horne,1,0,0,0,1,0,0,...,0,0,0,0,0,NaN,NaN,1996.0,1996.0,53.0
1,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,0YrtvWJMgSdVrk3SfNjTbx,Death Cab for Cutie,4,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2006.0,2006.0,NaN
2,00f82af1-6537-4d4b-9e03-103fffe19999,67FFKYikvTlvsPNk4NPOYJ,Baha Men,1,1,0,0,0,0,0,...,0,0,0,0,0,2001.0,2001.0,NaN,NaN,NaN
3,0103c1cc-4a09-4a5d-a344-56ad99a77193,0p4nmQO2msCgU4IF37Wi3j,Avril Lavigne,2,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2003.0,2003.0,NaN
4,012151a8-0f9a-44c9-997f-ebd68b5389f9,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,2,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2014.0,2014.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,fe85367e-4036-43c1-874b-b91af81cb4f3,68d6ZfyMUYURol2y15Ta2Y,Snarky Puppy,3,3,0,0,0,0,0,...,0,0,0,1,1,2016.0,2021.0,NaN,NaN,52.0
709,ff6e677f-91dd-4986-a174-8db0474b1799,3GBPw9NK25X1Wt2OUvOwY3,Jack Johnson,1,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2006.0,2006.0,NaN
710,ff95eb47-41c4-4f7f-a104-cdc30f02e872,1vgSaC0BPlL6LEm4Xsx59J,Brian Eno,1,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2010.0,2010.0,61.0
711,ffb5f70c-cd10-48a5-92d0-cea440e6fd40,3fLBmhcgWkPI47LfVQ8paB,Tamar Braxton,1,0,0,0,0,0,0,...,0,0,0,0,0,NaN,NaN,2014.0,2014.0,58.0


In [ ]:
# add genre info
df = with_mbz_artist_metadata(df)

In [ ]:
# features
# - chart_status  => [0: never charts, 1: has charted, 2: charted top 10]
# - award_status  => [0: never nominated, 1: nominated, 2: won]
# - artist_status => [0: never nominated, 1: previously nominated, 2: previous winner]

def chart_status(r):
    # should chart status account for album or artist charting?
    return 2 if r["position_min"] <= 25\
        else 1 if r["position_min"] > 0\
        else 0


def award_status(r):
    # should award status account for album or artist winning?
    return 2 if r["recording_award_winner"] >= 1\
        else 1 if r["recording_award_nominee"] >= 1\
        else 0


def artist_status(r):
    return 2 if r["artist_previous_wins"] >= 1\
        else 1 if r["artist_previous_nominations"] >= 1\
        else 0


df["chart_status"] = df.apply(chart_status, axis=1)
df["award_status"] =  df.apply(award_status, axis=1)
df["artist_status"] = df.apply(artist_status, axis=1)

In [ ]:
df = df[df["year"] >= 2000]

track_features_full_df = df[[
    'id',
    'name',
    'album',
    'album_id',
    'artists',
    'artist_ids',
    'track_number',
    'disc_number',
    'explicit',
    'duration_ms',
    'year',
    'release_date',
    'popularity',
    'isrc',
    # spotify audio features
    'audio_features', # has audio features
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    # artist
    'artist_genre',
    'artist_gender',
    'artist_country',
    'artist_hometown',
    'artist_begin',
    'artist_total_releases'
    # charts2000
    'months', # num months on charts
    'indicativerevenue',
    'position_min',
    'position_mean',
    'position_median',
    # grammy award result for this song
    'recording_award_category',
    'recording_award_nominee',
    'recording_award_winner',
    # grammy award result for this song
    'album_award_category',
    'album_award_nominee',
    'album_award_winner',
    # grammy nomination history for artist
    'artist_total_nominations',
    'artist_total_wins',
    'artist_first_nomination',
    'artist_first_win',
    'artist_previous_nominations',
    'artist_previous_wins',
    # quick categories
    'chart_status',
    'award_status',
    'artist_status'
]]

track_features_full_df.to_csv("../../data/02-track_features_balanced_wide.csv.gz", compression="gzip", index=False)
track_features_full_df.to_parquet("../../data/02-track_features_balanced_wide.pq", index=False)

track_features_slim_df = df[[
    'id',
    'name',
    'album',
    'album_id',
    'artists',
    'artist_ids',
    'track_number',
    'disc_number',
    'explicit',
    'duration_ms',
    'year',
    'release_date',
    'popularity',
    # spotify audio features
    'audio_features', # has audio features
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    # artist
    'artist_genre',
    'artist_gender',
    'artist_country',
    'artist_hometown',
    'artist_begin',
    'artist_total_releases',
    # quick categories
    'chart_status',
    'award_status',
    'artist_status'
]]

track_features_slim_df.to_csv("../../data/02-track_features_balanced.csv.gz", compression="gzip", index=False)
track_features_slim_df.to_parquet("../../data/02-track_features_balanced.pq", index=False)

In [ ]:
print(f"no. tracks: {len(df)}")
print(f"no. chart2000 songs: {len(song_chart_df)}")
print(f"no. spotify chart songs: {len(spotify_chart_df)}")
print(f"no. grammy songs: {len(recording_nominations_df)}")
print(f"no. tracks w/out features: {(track_features_df['audio_features'] == 0).sum()}")
print(f"no. track features w/out nominations: {track_features_df['spotify_track_id'].isnull().sum()}")
print(f"no. tracks w/out chart position: {len(track_features_df[(track_features_df['months'].isnull()) & (track_features_df['recording_award_nominee'] > 0)])}")
print(f"no. charted songs w/ nominations: {len(track_features_df[(~track_features_df['months'].isnull()) & (track_features_df['recording_award_nominee'] > 0)])}")
print(f"no. songs w/out nomination or chart: {len(track_features_df[(~track_features_df['months'].isnull()) & (track_features_df['recording_award_nominee'] == 0)])}")

# chart_status  => [0: never charts, 1: has charted, 2: charted top 25]
# artist_status => [0: never nominated, 1: previously nominated, 2: previous winner]
# award_status  => [0: never nominated, 1: nominated, 2: won]
df.groupby(["chart_status", "award_status", "artist_status"]).count()

In [ ]:
df.groupby(["chart_status", "award_status", "artist_status"]).aggregate({"id": "count"})